In [5]:
import sqlite3
import hashlib
import os

# Connect to SQLite database 
conn = sqlite3.connect('application_system.db')
cursor = conn.cursor()

# Create a new table for users
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL
);
''')
conn.commit()

In [6]:
import os
import hashlib

def hash_password(password):
    """Generate a salt and hash for the password."""
    salt = os.urandom(16)  
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'), 
                                  salt, 100000)
    pwdhash = pwdhash.hex()
    return salt.hex() + ':' + pwdhash

def verify_password(stored_password, user_password):
    """Verify a stored password against one provided by user"""
    
    # Check if the stored password has the expected format
    if ':' not in stored_password:
        print("Error: Stored password format is incorrect.")
        return False

    salt, pwdhash = stored_password.split(':')
    salt = bytes.fromhex(salt)  
    
    new_hash = hashlib.pbkdf2_hmac('sha512', 
                                   user_password.encode('utf-8'), 
                                   salt, 
                                   100000).hex()

    return pwdhash == new_hash

In [7]:
def register():
    username = input("Enter username: ")
    password = input("Enter password: ")  # getpass.getpass() is recommended here but might not work in Jupyter.
    
    hashed_password = hash_password(password)

    try:
        cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_password,))
        conn.commit()
        print("User created successfully.")
    except sqlite3.IntegrityError:
        print("Error: Username already exists.")

def login():
    username = input("Enter username: ")
    provided_password = input("Enter password: ")  # Consider using getpass for more security.

    cursor.execute("SELECT password FROM users WHERE username = ?", (username,))
    user = cursor.fetchone()

    if user is None:
        print("Username not found.")
        return False

    stored_password = user[0]

    if verify_password(stored_password, provided_password):
        print("Login successful.")
        return True
    else:
        print("Incorrect password.")
        return False

In [ ]:
# Credit card application function
def credit_card_application():
    print("------ Credit Card Application ------")

    # Gather user input
    name = input("Please enter your full name: ")
    annual_income = float(input("Enter your annual income: "))
    credit_score = int(input("Enter your credit score (300-850): "))
    employment_status = input("Are you currently employed? (yes/no): ")
    requested_credit_limit = float(input("Requested credit limit: "))

    # Business logic for credit approval
    if credit_score < 600:
        print(f"Sorry {name}, your credit score is too low.")
        return
    if employment_status.lower() != "yes":
        print(f"Sorry {name}, you need to be employed.")
        return
    if requested_credit_limit > annual_income * 0.5:
        print(f"Sorry {name}, your requested credit limit exceeds the permissible limit based on your income.")
        return

    print(f"Congratulations {name}! Your credit card application has been approved.")

while True:
    action = input("Do you want to login, register, or exit? (login/register/exit): ").lower()
    if action == "login":
        if login():
            credit_card_application()
            break  # Exit the loop after successful login and application completion
    elif action == "register":
        register()
    elif action == "exit":
        break
    else:
        print("Invalid action. Please enter 'login', 'register', or 'exit'.")

conn.close() 
